# Merging Data avec Pandas 

Dans la science de données on a souvent besoin de fusionner des tableaux. On peut le faire avec la fonction `merge()` qui prend comme paramètres deux `DataFrames` ainsi que les colonnes avec lesquelles réaliser les jointures. Cette méthode est très similaire à SQL sur le principe 🔭


### Importer pandas et les dataset `user_device.csv`, `user_usage.csv` et `android_devices.csv`

In [59]:
#1)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as ma
user_device=pd.read_csv('./data/user_device.csv')
user_usage=pd.read_csv('./data/user_usage.csv')
android_devices=pd.read_csv('./data/android_devices.csv')

### Importer les dataset & renommer la colonne `Branding` du dataset `android_devices.csv`

In [60]:
android_devices.rename(columns={'Retail Branding': 'Branding'}, inplace=True)

### Afficher les dataset 

In [61]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [63]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [64]:
android_devices.head()

,Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


### Votre premier merge

In [81]:
m1=pd.merge(user_usage,user_device[['use_id','platform','device']],on='use_id')
m1.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


### Afficher la `shape` de vos dataset ainsi que celle du dataset de sortie 

Que remarquez vous ? 

In [105]:
print("user_device dimension :{}".format(user_device[['use_id','platform','device']].shape))
print("user_usage dimsension :{}".format(user_usage.shape))
print("result dimension : {}".format(m1.shape))
#On remarque que le merge possède moins de lignes que les DF dont il est la fusion = logique car il ne reste que les itérations/données avec le même use_id dans les deux DF

user_device dimension :(272, 3)
user_usage dimsension :(240, 4)
result dimension : (159, 6)


### Afficher via `value_counts` les `use_id` présent dans le nouveau dataset ainsi que ceux non présent 

In [69]:
user_usage['use_id'].isin(m1['use_id']).value_counts()


True     159
False     81
Name: use_id, dtype: int64

### Le left merge

Afficher la `shape` du dataset `user_usage`, celle du dataset de sortie ainsi que les valeurs manquantes. 

In [94]:
print("user_usage dimensions :{}".format(user_usage.shape))
m2=pd.merge(user_usage,user_device[['use_id','platform','device']],on='use_id',how='left')
print("result dimensions :{}".format(m2.shape))
print("There are {} missing values in the result".format(m2['platform'].isna().sum()))

user_usage dimensions :(240, 4)
result dimensions :(240, 6)
There are 81 missing values in the result


### Afficher votre dataset 

In [95]:
m2.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


### Le right merge

Afficher la `shape` du dataset `user_device`, celle du dataset de sortie ainsi que les valeurs manquantes des colonnes `monthly_mb` et `platform`. 

In [97]:
m3=pd.merge(user_usage,user_device[['use_id','platform','device']],on='use_id',how='right')
print("user_device dimensions :{}".format(user_device.shape))
print("result dimensions :{}".format(m3.shape))
print("There are {} missing values in the result".format(m3['monthly_mb'].isna().sum()))
print("There are {} missing values in the result".format(m3['platform'].isna().sum()))

user_device dimensions :(272, 6)
result dimensions :(272, 6)
There are 113 missing values in the result
There are 0 missing values in the result


### Le outer merge example

Afficher les valeurs unique de `use_id` des datasets `user_device` & `user_usage`, celle du dataset de sortie ainsi que les valeurs no manquantes. 

In [111]:
print("There are {} unique values of use_id in our dataframes.".format(
        pd.concat([user_usage['use_id'], user_device['use_id']]).unique().shape[0]))
m4 = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id', how='outer', indicator=True)

print("Outer merge result has {} rows.".format(m4.shape))

print("There are {} rows with no missing values.".format(
    (m4.apply(lambda x: x.isnull().sum(), axis=1) == 0).sum()))

There are 353 unique values of use_id in our dataframes.
Outer merge result has (353, 7) rows.
There are 159 rows with no missing values.


### Afficher les lignes `0,1,200,201,350,351`

In [112]:
m4.iloc[[0, 1, 200,201, 350,351]]

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,_merge
0,21.97,4.82,1557.33,22787,android,GT-I9505,both
1,1710.08,136.88,7267.55,22788,android,SM-G930F,both
200,28.79,29.42,3114.67,23988,NaN,NaN,left_only
201,616.56,99.85,5414.14,24006,NaN,NaN,left_only
350,NaN,NaN,NaN,23050,ios,"iPhone7,2",right_only
351,NaN,NaN,NaN,23051,ios,"iPhone7,2",right_only


### Ajouter les colonnes `device` & `manufacturer`

In [136]:
# First, add the platform and device to the user usage.
m5 = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id',
                 how='left')

# Now, based on the "device" column in result, match the "Model" column in devices.
android_devices.rename(columns={"Retail Branding": "manufacturer"}, inplace=True)
m5 = pd.merge(m5, 
                  android_devices[['manufacturer', 'Model']],
                  left_on='device',
                  right_on='Model',
                  how='left')

m5.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,Model
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F


### Afficher les `device` commencant par 'GT'

In [130]:
android_devices[android_devices.Device.str.startswith('GT')]


,manufacturer,Marketing Name,Device,Model
1095,Bitmore,GTAB700,GTAB700,NID_7010
1096,Bitmore,GTAB900,GTAB900,S952
2402,Grundig,GTB1050,GTB1050,GTB 1050
2403,Grundig,GTB850,GTB850,GTB 850
2404,Grundig,TC69CA2,GTB801,GTB 801
...,...,...,...,...
10821,Samsung,Galaxy Y Pro,GT-B5510L,GT-B5510L
10822,Samsung,Galaxy Y Pro Duos,GT-B5512,GT-B5512
10823,Samsung,Galaxy Y Pro Duos,GT-B5512B,GT-B5512B
10824,Samsung,Galaxy Y TV,GT-S5367,GT-S5367


### Afficher le dataset des résultats 

In [131]:
m5.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


### Grouper vos données par `manufacturer`
Compter les `use_id` et afficher les moyennes des colonnes `outgoing_mins_per_month`, `outgoing_sms_per_month`, `monthly_mb`

In [137]:
m5.groupby("manufacturer").agg({
        "outgoing_mins_per_month": "mean",
        "outgoing_sms_per_month": "mean",
        "monthly_mb": "mean",
        "use_id": "count"
    })

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
manufacturer,,,,
HTC,299.842955,93.059318,5144.077955,44
Huawei,81.526667,9.500000,1561.226667,3
LGE,111.530000,12.760000,1557.330000,2
Lava,60.650000,261.900000,12458.670000,2
Lenovo,215.920000,12.930000,1557.330000,2
Motorola,95.127500,65.666250,3946.500000,16
OnePlus,354.855000,48.330000,6575.410000,6
Samsung,191.010093,92.390463,4017.318889,108
Sony,177.315625,40.176250,3212.000625,16
